In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.8 MB/s eta 0:00:00


In [2]:
# Bernstein–Vazirani Algorithm using Qiskit 2.x
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle(qc, inputs, ancilla, s):
    """Implements oracle for f(x) = s · x (no constant b)."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    fig = plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most

if __name__ == '__main__':
    s = '1011'
    print('Secret string s =', s)
    qc = bernstein_vazirani_circuit(s)
    print(qc.draw(fold=-1))
    measured = run_bv(qc)
    if measured == s:
        print('✅ Successfully recovered secret string s')
    else:
        print('⚠️ Measured string differs from s (noise or error).')


Secret string s = 1011
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101
⚠️ Measured string differs from s (noise or error).


**TASK-1**

In [5]:
# --- Imports ---
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# --- Oracle ---
def bv_oracle(qc, inputs, ancilla, s):
    """Implements oracle for f(x) = s · x"""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

# --- BV Circuit ---
def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Prepare |1⟩ on ancilla
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    bv_oracle(qc, inputs, ancilla, s)

    # Step 4: Apply Hadamard to input qubits only
    for q in inputs:
        qc.h(q)

    # Step 5: Measure input qubits
    qc.measure(inputs, creg)
    return qc

# --- Run BV ---
def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()

    most = max(counts, key=counts.get)
    print("Most frequent measurement:", most)
    return most

# --- Test ---
s = "1011"
print(f"Secret string s = {s}")
qc = bernstein_vazirani_circuit(s)
print(qc.draw(fold=-1))
measured = run_bv(qc)

if measured == s:
    print("✅ Successfully recovered secret string s")
else:
    print("⚠️ Measured string differs from s")


Secret string s = 1011
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Most frequent measurement: 1101
⚠️ Measured string differs from s


**TASK-2**

In [6]:
def bv_oracle_with_b(qc, inputs, ancilla, s, b):
    """Implements oracle for f(x) = s·x ⊕ b"""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    if b == 1:
        qc.x(ancilla)


In [7]:
def bernstein_vazirani_with_b(s, b):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)

    bv_oracle_with_b(qc, inputs, ancilla, s, b)

    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

# Test
s, b = "1011", 1
qc_b = bernstein_vazirani_with_b(s, b)
print(qc_b.draw(fold=-1))
measured = run_bv(qc_b)


     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ ├───┤ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫─┤ X ├─╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║ └───┘ ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Most frequent measurement: 1101


**TASK-3**

In [9]:
from qiskit.quantum_info import Statevector
import numpy as np

# Modified builder (no measurement)
def bernstein_vazirani_with_b_nom(circuit_secret, b):
    n = len(circuit_secret)
    qc = QuantumCircuit(n + 1)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(range(n + 1))
    bv_oracle_with_b(qc, inputs, ancilla, circuit_secret, b)
    qc.h(inputs)
    # no measurement here!
    return qc

# Build circuits for b=0 and b=1
qc_b0 = bernstein_vazirani_with_b_nom("1011", 0)
qc_b1 = bernstein_vazirani_with_b_nom("1011", 1)

# Simulate their statevectors
sv0 = Statevector.from_instruction(qc_b0)
sv1 = Statevector.from_instruction(qc_b1)

# Compare: should differ only by a global phase factor
ratios = [b/a for a, b in zip(sv0.data, sv1.data) if abs(a) > 1e-9]
unique = np.unique(np.round(ratios, 8))
print("Unique global phase ratio:", unique)


Unique global phase ratio: [-1.+0.j]


**TASK-4**

In [12]:
# --- Basic BV circuit (no constant bit) ---
from qiskit import QuantumRegister, ClassicalRegister, transpile, QuantumCircuit

def bv_oracle(qc, inputs, ancilla, s):
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1)
    creg = ClassicalRegister(n)
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    qc.h(inputs)
    qc.measure(inputs, creg)
    return qc

# --- Noise Model ---
def make_noise_model(p1=0.001, p2=0.02):
    nm = NoiseModel()
    nm.add_all_qubit_quantum_error(depolarizing_error(p1, 1), ['h', 'x'])
    nm.add_all_qubit_quantum_error(depolarizing_error(p2, 2), ['cx'])
    return nm

# --- Run simulation ---
def run_bv_noisy(qc, noise_model=None, shots=2048):
    sim = AerSimulator(noise_model=noise_model)
    tqc = transpile(qc, sim, optimization_level=1)
    job = sim.run(tqc, shots=shots)
    counts = job.result().get_counts()
    print("Counts:", counts)
    fig = plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print("Most frequent:", most)
    return counts

# --- Test run ---
s = "1011"
nm = make_noise_model(p1=0.002, p2=0.05)
qc = bernstein_vazirani_circuit(s)

print("Running noisy BV for secret:", s)
counts = run_bv_noisy(qc, nm)


Running noisy BV for secret: 1011
Counts: {'0000': 28, '0001': 57, '1000': 1, '1001': 23, '1100': 30, '0101': 78, '1101': 1831}
Most frequent: 1101


**TASK-5**

In [13]:
pip install qiskit-ibm-runtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.1 MB/s eta 0:00:00


In [14]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session

# Make sure your account is saved:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="YOUR_TOKEN")

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

qc_real = bernstein_vazirani_circuit("1011")

with Session(service=service, backend=backend) as session:
    sampler = Sampler(session=session)
    tqc = transpile(qc_real, backend)
    job = sampler.run([tqc])
    quasi = job.result().quasi_dists[0]
    counts = {k: int(v * 1024) for k, v in quasi.items()}

plot_histogram(counts)
plt.show()



AccountNotFoundError: "Unable to find account. Please make sure an account with the channel name 'ibm_quantum_platform' is saved."